# Custom networks

In previous tutorials, we used the pre-set networks offered by the library. These networks are practical because they are ready to be configured. However, you may need to develop other forms of network than those available. That's why sumo-experiments offers a `components` subpackage that lets you create networks yourself (and with which the pre-set networks are designed).

To create an experiment, you need at least one function returning a `FlowBuilder` object and another function returning an `InfrastructureBuilder` object. These two objects are located in the `components` package.

In this example, we will create a network with a triangle shape. Each angle of the triangle will be connected to an edge that will serve as an entry and an exit for the vehicles.


## Custom infrastructures

First, we need to define a function that return an `InfrastructureBuilder` object. To be complete, a set of infrastructures must contain nodes, edges, edge type, connections and traffic light programs. Each of this element can be added with a method of `InfrastructureBuilder` : `add_node`, `add_edge`, `add_edge_type`, `add_connection` and `add_traffic_light_program`.

To define a node, you need x-coordinate and y-coordinate. There is another parameter `type` that define the type of the node. Leave it empty if the node is a normal node, and fill it with *"traffic_light"* if it's a traffic light. In this cas, you must add the id of a traffic light program.

Then you need to define an edge type. An edge type represent the characteritics of an edge : number of lanes, speed, etc. To define an edge type, you need to define each parameter into a dictionnary and pass it to `add_edge_type` with the parameter `params`. You can find a full list of available parameters [here](https://sumo.dlr.de/docs/SUMO_edge_type_file.html).

You can add edges now. An edges is a connection between two nodes, __in one way only__. If you want to create a two-way road between a point *A* and a point *B*, you need to create two edges : *A*->*B* and *B*->*A*. The `edge_type` parameter must be filled with the id of an edge type.

A connection is a relationship between two connected edges. If a connection is set between two edges, vehicles can pass from one edge to the other. Be sure that the edges are connected __in the right way__. The first edge must end by a node *A* and the other end must start from this same node. For example, if you have three aligned points *A*, *B* and *C*, and edges connecting *A* to *B* and *B* to *C*, you must connect the edges *A*->*B* and *B*->*C* (and *C*->*B* to *B*->*A* in the other way). If you try to connect *B*->*A* and *B*->*C*, an *interesting behaviour* will occur.

A traffic light program is the fixed setting for a traffic light. You __can't__ define one program for multiple traffic lights. The `phases` parameter mus be a list of dictionnary, where each dictionnary represents a different phase of the traffic light. During the simulation, SUMO will iterate over all phases, and repeat it until the end. A phase is composed of two elements : a duration and a traffic light configuration. The duration is set with the key `duration` in the dictionnary, and the configuration with the key `state`. A configuration is set with a string, and must contain the state of traffic light for each possible direction that a vehicle can take at the intersection. You will find more informations about it [here](https://sumo.dlr.de/docs/Simulation/Traffic_Lights.html).

Let's see an example with the triangle network that we want to build.

In [1]:
from sumo_experiments.components import InfrastructureBuilder

def triangle_infrastructures(config):
    
    # Instanciating object
    infra = InfrastructureBuilder()
    
    # Adding nodes for the triangle
    infra.add_node(id='left_angle', x=0, y=0, type='traffic_light', tl_program='left_tl') # left angle
    infra.add_node(id='right_angle', x=100, y=0, type='traffic_light', tl_program='right_tl') # right angle
    infra.add_node(id='top_angle', x=50, y=86.6, type='traffic_light', tl_program='top_tl') # top angle
    
    # Adding nodes for the entry/exit edges
    infra.add_node(id='entry_exit_left_angle', x=-100, y=0) # to the left angle
    infra.add_node(id='entry_exit_right_angle', x=200, y=0) # to the right angle
    infra.add_node(id='entry_exit_top_angle', x=50, y=186.6) # to the top angle
    
    # Adding edge type
    infra.add_edge_type(id='default_edge_type', params={'numLanes': '1', 'speed': 50})
    
    # Adding edges between nodes / Don't forget to create it in the two ways !
        # Between left and right angles
    infra.add_edge(id='left_right', from_node='left_angle', to_node='right_angle', edge_type='default_edge_type')
    infra.add_edge(id='right_left', from_node='right_angle', to_node='left_angle', edge_type='default_edge_type')
        # Between left and top angles
    infra.add_edge(id='left_top', from_node='left_angle', to_node='top_angle', edge_type='default_edge_type')
    infra.add_edge(id='top_left', from_node='top_angle', to_node='left_angle', edge_type='default_edge_type')
        # Between right and top angles
    infra.add_edge(id='right_top', from_node='right_angle', to_node='top_angle', edge_type='default_edge_type')
    infra.add_edge(id='top_right', from_node='top_angle', to_node='right_angle', edge_type='default_edge_type')
        # Between left angle and entry/exit
    infra.add_edge(id='left_exit', from_node='left_angle', to_node='entry_exit_left_angle', edge_type='default_edge_type')
    infra.add_edge(id='entry_left', from_node='entry_exit_left_angle', to_node='left_angle', edge_type='default_edge_type')
        # Between right angle and entry/exit
    infra.add_edge(id='right_exit', from_node='right_angle', to_node='entry_exit_right_angle', edge_type='default_edge_type')
    infra.add_edge(id='entry_right', from_node='entry_exit_right_angle', to_node='right_angle', edge_type='default_edge_type')
        # Between top angle and entry/exit
    infra.add_edge(id='top_exit', from_node='top_angle', to_node='entry_exit_top_angle', edge_type='default_edge_type')
    infra.add_edge(id='entry_top', from_node='entry_exit_top_angle', to_node='top_angle', edge_type='default_edge_type')
    
    
    # Adding connections
        # Between triangle edges, clockwise
    infra.add_connection(from_edge='left_top', to_edge='top_right')
    infra.add_connection(from_edge='top_right', to_edge='right_left')
    infra.add_connection(from_edge='right_left', to_edge='left_top')
        # Between triangle edges, counter-clockwise
    infra.add_connection(from_edge='top_left', to_edge='left_right')
    infra.add_connection(from_edge='left_right', to_edge='right_top')
    infra.add_connection(from_edge='right_top', to_edge='top_left')
        # Between left entry/exit edges and others
    infra.add_connection(from_edge='entry_left', to_edge='left_right')
    infra.add_connection(from_edge='entry_left', to_edge='left_top')
    infra.add_connection(from_edge='right_left', to_edge='left_exit')
    infra.add_connection(from_edge='top_left', to_edge='left_exit')
        # Between right entry/exit edges and others
    infra.add_connection(from_edge='entry_right', to_edge='right_left')
    infra.add_connection(from_edge='entry_right', to_edge='right_top')
    infra.add_connection(from_edge='left_right', to_edge='right_exit')
    infra.add_connection(from_edge='top_right', to_edge='right_exit')
        # Between top entry/exit edges and others
    infra.add_connection(from_edge='entry_top', to_edge='top_left')
    infra.add_connection(from_edge='entry_top', to_edge='top_right')
    infra.add_connection(from_edge='left_top', to_edge='top_exit')
    infra.add_connection(from_edge='right_top', to_edge='top_exit')
    
    
    # Adding traffic light program for each traffic light
    infra.add_traffic_light_program(id='left_tl',
                                      phases=[{'duration': 30, 'state': 'rrGGGG'},
                                              {'duration': 3, 'state': 'rryyyy'},
                                              {'duration': 30, 'state': 'GGrrrr'},
                                              {'duration': 3, 'state': 'yyrrrr'}])
    
    infra.add_traffic_light_program(id='right_tl',
                                      phases=[{'duration': 30, 'state': 'GGGGrr'},
                                              {'duration': 3, 'state': 'yyyyrr'},
                                              {'duration': 30, 'state': 'rrrrGG'},
                                              {'duration': 3, 'state': 'rrrryy'}])
    
    infra.add_traffic_light_program(id='top_tl',
                                      phases=[{'duration': 20, 'state': 'GGrrrr'},
                                              {'duration': 3, 'state': 'yyrrrr'},
                                              {'duration': 20, 'state': 'rrGGrr'},
                                              {'duration': 3, 'state': 'rryyrr'},
                                              {'duration': 20, 'state': 'rrrrGG'},
                                              {'duration': 3, 'state': 'rrrryy'}])
    
    return infra

***Note :** The function must contain one and only one parameter (generally `config`) that will be in a dictionnary format. This `config` contains all of the parameter declared with the `set_parameter()` method of Experiment. Config parameters can then be used within the function that we declared. We'll see a short example in the custom flows part, that works in the same way.*

## Custom flows

Creating custom flows follows the same principle than the infrastructures. We must create a function (with a `config` parameter) that returns a `FlowBuilder` object, from the `component` subpackage.

The `FlowBuilder` object has three methods :
- The `add_v_type` creates a vehicle type that can be used to generate a flow.
- The `add_route` is a deprecated method that was supposed to define a route that will be used to define a flow.
- The `add_flow` method generates flows between two edges, with a defined frequency, v_type, beginning, ending and distribution. The distribution can be 'uniform' or 'binomial'. With 'uniform', vehicles will be generated following a Uniform Law, wereas with 'binomial', vehicles will be generated following a Binomial Law.

Since it's pretty much the same principle as for InfrastructureBuilder, we'll focus on the use of the `config` parameter. As we said previously, the `config` parameter will contain all the parameter set with the `set_parameter` method from Experiment. This parameters can then be used with the `config` dictionnary directly in our custom functions. Let's see an example here : we want to define equal flows for the triangle network. Flows will enter from each entry and have a __frequency__. We want this frequency to be customizable. Let's see how.

In [2]:
from sumo_experiments.components import FlowBuilder

def triangle_flows(config):
    
    # Instanciating object
    flows = FlowBuilder()
    
    # Adding v_type
    flows.add_v_type(id='car0') # Default car in SUMO.
    
    # Adding flows from left entry
    flows.add_flow(id='f_left_top', 
                   from_edge='entry_left', 
                   to_edge='top_exit', 
                   begin=0, end=1000, 
                   frequency=config['frequency'], 
                   v_type='car0', distribution="binomial")
    flows.add_flow(id='f_left_right', 
                   from_edge='entry_left', 
                   to_edge='right_exit', 
                   begin=0, end=1000, 
                   frequency=config['frequency'], 
                   v_type='car0', distribution="binomial")
    
    # Adding flows from right entry
    flows.add_flow(id='f_right_top', 
                   from_edge='entry_right', 
                   to_edge='top_exit', 
                   begin=0, end=1000, 
                   frequency=config['frequency'], 
                   v_type='car0', distribution="binomial")
    flows.add_flow(id='f_right_left', 
                   from_edge='entry_right', 
                   to_edge='left_exit', 
                   begin=0, end=1000, 
                   frequency=config['frequency'], 
                   v_type='car0', distribution="binomial")
    
    # Adding flows from top entry
    flows.add_flow(id='f_top_left', 
                   from_edge='entry_top', 
                   to_edge='left_exit', 
                   begin=0, end=1000, 
                   frequency=config['frequency'], 
                   v_type='car0', distribution="uniform")
    flows.add_flow(id='f_top_right', 
                   from_edge='entry_top', 
                   to_edge='right_exit', 
                   begin=0, end=1000, 
                   frequency=config['frequency'], 
                   v_type='car0', distribution="uniform")
    
    return flows

Here, we use a `frequency` value from config to configure the frequency of the different flows. Flows from left and right will be generated with a Binomial Law, wereas flows from top will be generated with an Uniform Law, to show you the difference.

***Note :** With this kind of implementation for the `frequency` (with no default value), the parameter __must__ be defined in the experiment. Otherwise, the triangle function will generate error.*

## Use custom networks

Custom networks are used exactly like pre-set networks. We create an Experiment, and fill the parameters with the custom functions.

In [3]:
from sumo_experiments import Experiment

exp = Experiment(
    name='triangle',
    infrastructures=triangle_infrastructures,
    flows=triangle_flows
)

exp.set_parameter('frequency', 100)

exp.run(gui=True)

exp.clean_files()

Success.
Success.


## Optional : Custom detectors

The sumo-experiments library also allows to implement detectors. This detectors will be used by the TraCi module from SUMO, that allows you to control the infrastructures of a network simulation step by simulation step, and so to implement adaptative behaviours. This side of SUMO will be the subject of the next tutorial.

Detectors are generated exactly like infrastructures and flows : by a function using a `config` parameter, and returning a `DetectorBuilder` object.

A detector in sumo-experiments is a SUMO laneAreaDetector. It needs two elements to be defined :
- A __lane__ (and not an edge)
- A position on the lane, in [0, lane_length]

Let's add detectors to the previous network. We're going to add short range detectors, that can only detect one car.

***Note :** Find the right position for detectors is kind of tricky. For now, you have to tinker to find the right settings.*

In [6]:
from sumo_experiments.components import DetectorBuilder

def triangle_detectors(config):
    
    # Instanciate the object
    det = DetectorBuilder()
    
    # Add detectors to the left intersection
    det.add_lane_area_detector(id='detector_left_left',
                            lane='entry_left_0', # edge_[lane_id]
                            pos=(81 - 8 + 13)) # lane length - detector length + offset due to intersection
    det.add_lane_area_detector(id='detector_left_right',
                            lane='right_left_0',
                            pos=(81 - 8))
    det.add_lane_area_detector(id='detector_left_top',
                            lane='top_left_0',
                            pos=(81 - 8))
    
    # Add detectors to the right intersection
    det.add_lane_area_detector(id='detector_right_right',
                            lane='entry_right_0', # edge_[lane_id]
                            pos=(81 - 8 + 13)) # lane length - detector length + offset due to intersection
    det.add_lane_area_detector(id='detector_right_left',
                            lane='left_right_0',
                            pos=(81 - 8))
    det.add_lane_area_detector(id='detector_right_top',
                            lane='top_right_0',
                            pos=(81 - 8))
    
    # Add detectors to the top intersection
    det.add_lane_area_detector(id='detector_top_top',
                            lane='entry_top_0', # edge_[lane_id]
                            pos=(81 - 8 + 13)) # lane length - detector length + offset due to intersection
    det.add_lane_area_detector(id='detector_top_left',
                            lane='left_top_0',
                            pos=(81 - 8))
    det.add_lane_area_detector(id='detector_top_right',
                            lane='right_top_0',
                            pos=(81 - 8))
    
    return det

In [7]:
from sumo_experiments import Experiment

exp = Experiment(
    name='triangle',
    infrastructures=triangle_infrastructures,
    flows=triangle_flows,
    detectors=triangle_detectors
)

exp.set_parameter('frequency', 100)

exp.run(gui=True)

exp.clean_files()

Success.
Success.
